# Demonstration of computational protocols for molecular QTL analyses

This notebook illustrates the computational protocols available from this repository for the detection and analysis of molecular QTLs (xQTLs). A minimal toy data-set consisting of 49 de-identified samples are used for the analysis.

## Analysis

Please visit [the homepage of the protocol website](https://cumc.github.io/xqtl-pipeline/) for the general background on this resource. To perform a complete analysis from molecular phenotype quantification to xQTL discovery, please conduct your analysis in the order listed below, each link contains a mini-protocol for a specific task. 

### Bulk RNA-seq molecular phenotype quantification

1. [Reference data munging](https://cumc.github.io/xqtl-pipeline/code/data_preprocessing/reference_data.html)
2. [Quantification of gene expression](https://cumc.github.io/xqtl-pipeline/code/molecular_phenotypes/bulk_expression.html)
3. [Quantification of alternative splicing events](https://cumc.github.io/xqtl-pipeline/code/molecular_phenotypes/splicing.html)

### xQTL association analysis

1. [Phenotype data munging](https://cumc.github.io/xqtl-pipeline/code/data_preprocessing/phenotype_preprocessing.html)
2. [Genotype data munging](https://cumc.github.io/xqtl-pipeline/code/data_preprocessing/genotype_preprocessing.html)
3. [Covariates data munging](https://cumc.github.io/xqtl-pipeline/code/data_preprocessing/covariate_preprocessing.html)
4. [cis-QTL association testing](https://cumc.github.io/xqtl-pipeline/code/association_scan/cisQTL_scan.html)
5. [trans-QTL association testing](https://cumc.github.io/xqtl-pipeline/code/association_scan/transQTL_scan.html)

## Data 

The input data and some of the intermediate output data can be download from [this Synapse folder](https://www.synapse.org/#!Synapse:syn36416601). To download the files, a synapse account and synapse clients are both needed. To setup a synapse client, please follow [this post](https://help.synapse.org/docs/Installing-Synapse-API-Clients.1985249668.html).

## Software Enviorments (To be updated)